## Practical [Necromancy](https://en.wikipedia.org/wiki/Necromancy)
We're going to resurrect Flinders Petrie.

_This demo is based on the colab notebook by Max Woolf (see his 2019 [blog post](https://minimaxir.com/2019/09/howto-gpt2/))_

Save a copy of this in your gdrive (file -> save) and then you can do whatever you want with it. Make sure the runtime is 'gpu - t4' or things'll be very slow indeed. (It should automatically have that runtime, btw).

We're using an older model because a) we don't always need the latest flashy thing b) the older model can be explored c) we should try understanding the foundational technologies before jumping to the latest flashy thing. A and C are really aspects of.the same argument I suppose.

In [ ]:
# but first we need some lego blocks
%%capture
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
gpt2.download_gpt2(model_name="124M") #124 million parameters! State of the art, only a few short years ago. Now, something that you can run in a notebook.

Then, assuming that the man's published works capture something meaningful about his worldview (if we can make that assumption):

In [ ]:
%%capture
!wget https://www.gutenberg.org/cache/epub/7386/pg7386.txt
!wget https://www.gutenberg.org/cache/epub/70049/pg70049.txt
!wget https://www.gutenberg.org/cache/epub/52570/pg52570.txt
!wget https://www.gutenberg.org/cache/epub/63311/pg63311.txt
!wget https://www.gutenberg.org/cache/epub/56095/pg56095.txt

In [ ]:
# join all of that together
# and set the file_name variable
!cat *.txt > petrie.txt
file_name = "petrie.txt"

Hey, it's all just data, right? You mean you're worried about representation? Comprehensiveness? Balance? Nah, bro, moar data will just do the trick.

Now we'll add another layer of culture, of memory, of voice on top of the 'frog dna' of the original model. If it was good enough for Jurassic Park, it's good enough for us.

optional-but-helpful parameters for gpt2.finetune:

+ restore_from: Set to fresh to start training from the base GPT-2, or set to latest to restart training from an existing checkpoint.
+ sample_every: Number of steps to print example output
+ print_every: Number of steps to print training progress.
+ learning_rate: Learning rate for the training. (default 1e-4, can lower to 1e-5 if you have <1MB input data)
+ run_name: subfolder within checkpoint to save the model. This is useful if you want to work with multiple models (will also need to specify run_name when loading the model)
+ overwrite: Set to True if you want to continue finetuning an existing model (w/ restore_from='latest') without creating duplicate copies.

This might take a while, even with the short-trained parameters I've set below. We'll check in later. Keep an eye on the 'loss' function. When that starts to flatline (the descent is no longer steep) the model is starting to over fit. [Chantal Brousseau has a good explanation at the Programming Historian](https://programminghistorian.org/en/lessons/interrogating-national-narrative-gpt#gradient-descent-explained).


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=100, # in a proper training session you'd run at least 1000
              restore_from='fresh',
              run_name='run1',
              print_every=10, # and you'd print out much less frequently
              sample_every=20, # and sample every once in a while
              save_every=50 # and probably save every 500
              ) # with these settings and this little data, you're going to get 'overfitting' - why is that a problem?

This resulting model is not a chat model. It does not have layers of human reinforcement learning. It does not have layers of instruction tuning. It has the base model, and then Flinders Petrie smeared all over the surface-most layers. (If you want to see just what Hapsburg-monstrosities most models are, check out ['Models All The Way Down'](https://knowingmachines.org/models-all-the-way) from [Knowing Machines](https://knowingmachines.org)).

**Warning** All the awfulness of the world is contained in these models, given their source data. And we've just grafted a 19th century colonialist archaeologist on top of it. Be warned - things could get unpleasant.

Treat it, maybe, like resurrecting a ghost. The ghost might make sense. It might speak in riddles. What it actually _means_ says more about you than it. Take it as a kind of _deformance_. Consider it [the inversion of the digital humanities](https://www.cyberneticforests.com/news/toward-a-multi-modal-media-theory).

Run the next block to see what percolates up from the latent spaces of Petrie's writing. You can change the string in line 3, between the quotes for the 'prefix' variable, to explore other ideas.

In [ ]:
gpt2.generate(sess,
              model_name='124M',
              prefix="Local workers are",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

Salvaggio:

> Generative AI is digital humanities run in reverse.[...]instead of saying, this is a new song that's been uploaded, and analyzing the wave patterns to determine that it has similarity to other songs that are, say, in the chillwave genre, and then tagging it chillwave, we do it backward. You type in chillwave, and the idea that it's used to rely on to define that piece of music is now being reversed. To say, we will trace the conventions that align with this classification and render that information as a waveform. It's a reversal.
>
> AI becomes a system for producing approximations of human media that align with all the data swept together to describe that media.
- Eryck Salvaggio, [https://www.cyberneticforests.com/news/toward-a-multi-modal-media-theory](https://www.cyberneticforests.com/news/toward-a-multi-modal-media-theory)

In [ ]:
## if you want to do another training run,
## 1. upload your training dataset
## 2. then restart the session (runtime -> restart session)
## 3. and rerun these imports.
## 4. then set your filename & run the training cell.

import gpt_2_simple as gpt2 #necessary when you have to restart the session to do another training run
from datetime import datetime #necessary when you have to restart the session to do another training run
sess = gpt2.start_tf_sess()

# gpt2.load_gpt2(sess, run_name='run1') # uncomment this line if you want to reload a previous training run, and give it the run name.